## File structure

First, download the scene-specific parameters from [Google Drive](https://drive.google.com/drive/folders/1jWjJDcpYJ6cNECV1USC0dg0xRwMTSRWb?usp=sharing).

In your PVDN dataset (day cycle) directory, the downloaded parameter config files have to be stored like this:
```
/path/to/PVDN/day/<split>/labels/kpbms_params
├── S*****.json
├── S*****.json
├── ....
├── S*****.json
```

For example, for the **val** split, this would look like this:
```
/path/to/PVDN/day/val/labels/kpbms_params
├── S00071.json
├── S00092.json
├── S00100.json
├── S00101.json
├── S00121.json
├── S00123.json
├── S00126.json
├── S00132.json
├── S00135.json
├── S00164.json
├── S00168.json
├── S00192.json
├── S00195.json
├── S00260.json
├── S00284.json
├── S00294.json
├── S00309.json
├── S00355.json
├── S00370.json
└── S00372.json
```

A scene config file looks like this (example: S00071):
```
{
    "kernel_size": 300,
    "lower_direct": 0.7000000000000001,
    "lower_indirect": 0.9,
    "n": 6,
    "selem_size": 13,
    "sigma": 0,
    "upper_direct": 1.0,
    "upper_indirect": 1.0
}
```

## Generate the dataset
The following code will generate the dataset. For each image, two saliency maps will be stored -- one for direct, one for indirect light instances. The file structure will look like this:

```
/home/lukas/Development/datasets/PVDN_test/day/<split>/saliency_maps/direct
├── S*****
    ├── ******.png
    ├── ******.png
    ├── ...
    ├── *****.png
├── S*****
    ├── ...
├── S*****
├── S00121
├── S00123
├── S00126
├── S00132
├── S00135
├── S00164
├── S00168
├── S00192
├── S00195
├── S00260
├── S00284
├── S00294
├── S00309
├── S00355
├── S00370
└── S00372

/home/lukas/Development/datasets/PVDN_test/day/<split>/saliency_maps/indirect
├── S*****
    ├── ******.png
    ├── ******.png
    ├── ...
    ├── *****.png
├── S*****
    ├── ...
├── S*****
├── S00121
├── S00123
├── S00126
├── S00132
├── S00135
├── S00164
├── S00168
├── S00192
├── S00195
├── S00260
├── S00284
├── S00294
├── S00309
├── S00355
├── S00370
└── S00372
```

### Run the code

**Note:** Make sure to adjust the dataset paths in the second cell accordingly.

In [ ]:
!pip install pip --upgrade
!pip install -e ..  # install our package

import os
from tqdm import tqdm

from torch.utils.data import DataLoader

from kpsaliency.datasets import CumulatedSaliencyMapDataset as SaliencyMapDataset
from kpsaliency.datasets import cumulated_collate_fn as collate_fn
from kpsaliency.generators import KPBMSGenerator

In [2]:
## setup all paths

dataset_dir = "/home/lukas/Development/datasets/PVDN/day"   # adapt this to your
                                    # specific path
splits = ("train", "val", "test")

assert os.path.exists(dataset_dir)
for split in splits:
    assert os.path.exists(os.path.join(dataset_dir, split))


In [ ]:
## run the dataset generation process

for split in splits:
    # setup and check paths
    split_dir = os.path.join(dataset_dir, split)
    params_dir = os.path.join(split_dir, "labels/kpbms_params")
    if not os.path.isdir(params_dir):
        raise NotADirectoryError(f"{params_dir} not found. Please check that you set"
                                 f" up the parameter directories properly.")

    # setup dataset
    dataset = SaliencyMapDataset(split_dir)

    # setup scene-specific generators
    generators = {
        s.split(".")[0]: KPBMSGenerator.from_json(os.path.join(params_dir, s))
        for s in os.listdir(params_dir)
    }

    # run generation
    print(split)
    dataset.generate_dataset(bms_generator=generators, n_workers=4)

## Using the `SaliencyMapDataset`-class for your pipelines

After you generated the dataset following the steps above, you can use the class for your experiment pipelines:

In [ ]:

dataset = SaliencyMapDataset(
    path=os.path.join(dataset_dir, "test")  # using test split in this example
)

dataloader = DataLoader(
    dataset=dataset,
    batch_size=4,
    shuffle=False,
    collate_fn=collate_fn   # note custom collate_fn
)

for imgs, smaps_direct, smaps_indirect, infos, vehicles in tqdm(dataloader):
    # do something here
    pass